# Model Training

### 1.1 importing packages

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [45]:
df = pd.read_csv('data/StudentsPerformance.csv')

In [46]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
df['Total_score'] = df['math score'] + df['reading score'] + df['writing score']
df['Average_score'] = df['Total_score'] /3 

In [47]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## Feature Engineering
- Making total score and average score feature
- Dropping 3 subject

In [7]:
df = df.drop(['math score', 'reading score', 'writing score'], axis = 1)

In [48]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## Preparing X(independent) and y(dependent) variables

In [49]:
X = df.drop(['math score'], axis = 1)
y = df['math score']

In [50]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [51]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [52]:
print("Categories in Gender variable:",end="" )
print(df['gender'].unique())

print("Categories in Race variable:",end="")
print(df['race/ethnicity'].unique())

print("Categories in Parental level of education variable:",end=" " )
print(df['parental level of education'].unique())

print("Categories in Lunch variable:",end="" )
print(df['lunch'].unique())

print("Categories in Test preparation course variable:",end="" )
print(df['test preparation course'].unique())

Categories in Gender variable:['female' 'male']
Categories in Race variable:['group B' 'group C' 'group A' 'group D' 'group E']
Categories in Parental level of education variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in Lunch variable:['standard' 'free/reduced']
Categories in Test preparation course variable:['none' 'completed']


## Columns Transformation for the Model

In [53]:
numerical_features = X.select_dtypes(exclude = "object").columns
categorical_features = X.select_dtypes(include = "object" ).columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", categorical_transformer, categorical_features),
        ("StandardScaler", numerical_transformer, numerical_features)
    ]
)

## Insight 
- A pipeline has been created to transform all the numerical and categorical columns 

In [54]:
X = preprocessor.fit_transform(X)

In [55]:
X.shape

(1000, 19)

## TrainTest Split

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

## Create an evaluate function to give all the metrics after model training

In [57]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [58]:
models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoosting Regressor" : CatBoostRegressor(verbose = False),
    "AdaBoost Regressor" : AdaBoostRegressor()
}

model_list = []
r2_list = []

for name, mdl in models.items():
    #model training
    mdl.fit(X_train, y_train)

    #prediction
    y_train_pred = mdl.predict(X_train)
    y_test_pred = mdl.predict(X_test)

    #Evaluate train and test dataset
    model_train_mae, model_train_rmse, model_train_r2_score = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2_score = evaluate_model(y_test, y_test_pred)

    model_list.append(name)
    r2_list.append(model_test_r2_score)

    print(f"\n===== {name} =====")

    print("Model performance for Training set")
    print("- Root Mean Squared Error(rmse) : {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error(mae) : {:.4f}".format(model_train_mae))
    print("- R2 Score : {:.4f}".format(model_train_r2_score))

    print("-------------------------------")

    print("Model performance for Test set")
    print("- Root Mean Squared Error(rmse) : {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error(mae) : {:.4f}".format(model_test_mae))
    print("- R2 Score : {:.4f}".format(model_test_r2_score))

    print('='*35)
    print('\n')
    
    


===== Linear Regression =====
Model performance for Training set
- Root Mean Squared Error(rmse) : 5.3240
- Mean Absolute Error(mae) : 4.2691
- R2 Score : 0.8743
-------------------------------
Model performance for Test set
- Root Mean Squared Error(rmse) : 5.3773
- Mean Absolute Error(mae) : 4.2053
- R2 Score : 0.8812



===== Lasso =====
Model performance for Training set
- Root Mean Squared Error(rmse) : 6.5938
- Mean Absolute Error(mae) : 5.2063
- R2 Score : 0.8071
-------------------------------
Model performance for Test set
- Root Mean Squared Error(rmse) : 6.5197
- Mean Absolute Error(mae) : 5.1579
- R2 Score : 0.8253



===== Ridge =====
Model performance for Training set
- Root Mean Squared Error(rmse) : 5.3233
- Mean Absolute Error(mae) : 4.2650
- R2 Score : 0.8743
-------------------------------
Model performance for Test set
- Root Mean Squared Error(rmse) : 5.3904
- Mean Absolute Error(mae) : 4.2111
- R2 Score : 0.8806



===== K-Neighbors Regressor =====
Model performa